In [109]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from sklearn.metrics import classification_report

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization, Flatten, Dense, Dropout, Activation

from skimage import io, transform, exposure

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from datetime import datetime
import os

# Load CSVs Tran and Test

In [35]:
df_Train = pd.read_csv("../dataset/archive/Train.csv")[["ClassId", "Path"]]
df_Test = pd.read_csv("../dataset/archive/Test.csv")[["ClassId", "Path"]]

In [36]:
df_Train

,ClassId,Path
0,20,Train/20/00020_00000_00000.png
1,20,Train/20/00020_00000_00001.png
2,20,Train/20/00020_00000_00002.png
3,20,Train/20/00020_00000_00003.png
4,20,Train/20/00020_00000_00004.png
...,...,...
39204,42,Train/42/00042_00007_00025.png
39205,42,Train/42/00042_00007_00026.png
39206,42,Train/42/00042_00007_00027.png
39207,42,Train/42/00042_00007_00028.png


In [67]:
total = df_Train.shape[0]+df_Test.shape[0]
print(f"Train Size: {df_Train.shape[0]} images ({df_Train.shape[0]/ total*100})")
print(f"Test Size: {df_Test.shape[0]} images ({df_Test.shape[0]/ total*100})")

Train Size: 39209 images (75.63610409151411)
Test Size: 12630 images (24.363895908485887)


# Function to Load Images 

In [113]:
# This function first shuffle the dataset, after load the image to memory, resize it, equelize the contrast and finally save to an array.
def load_images(main_path, df):
    data = []
    
    # SHUFFLE DATA FRAME
    df_shuffle = df.sample(frac=1)
    
    for (i, image_path) in enumerate(df):
        if i > 0 and i % 1000 == 0:
            print(f"[DEBUG] Total processed images: {i}")
            
        image_full_path = os.path.sep.join([main_path, image_path])
        image = io.imread(image_full_path)

        image = transform.resize(image, (32,32))
        image = exposure.equalize_adapthist(image, clip_limit=0.1)

        data.append(image)
            
    data = np.array(data)
    return data

# Model Function

In [69]:
def build(width, height, depth, classes):
        model = Sequential()

        inputShape = (height, width, depth)
        dim = 1

        model.add(Conv2D(8, (5,5), padding = "same", input_shape = inputShape))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=dim))
        model.add(MaxPooling2D(pool_size=(2,2)))

        model.add(Conv2D(16, (3,3), padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=dim))
        model.add(Conv2D(16, (3,3), padding = "same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=dim))
        model.add(MaxPooling2D(pool_size=(2,2)))

        model.add(Conv2D(32, (3,3), padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=dim))
        model.add(Conv2D(32, (3,3), padding = "same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=dim))
        model.add(MaxPooling2D(pool_size=(2,2)))

        model.add(Flatten())
        model.add(Dense(128))
        model.add(Activation("relu"))
        model.add(BatchNormalization())
        model.add(Dropout(0.5))

        model.add(Flatten())
        model.add(Dense(128))
        model.add(Activation("relu"))
        model.add(BatchNormalization())
        model.add(Dropout(0.5))

        model.add(Dense(classes))
        model.add(Activation("softmax"))

        return model

In [116]:
def run():
    NUM_EPOCHS = 30
    INIT_LR = 0.001
    BS = 64
    
    print(f"[DEBUG] Loading dataset ...\n")
    df_Train = pd.read_csv("../dataset/archive/Train.csv")[["ClassId", "Path"]]
    df_Test = pd.read_csv("../dataset/archive/Test.csv")[["ClassId", "Path"]]
    label_names = pd.read_csv("../dataset/archive/signnames.csv")["SignName"]
    
    print(f"[DEBUG] Load images from file and prepare data ...\n")
    X_Train = load_images("../dataset/archive", df_Train["Path"])
    X_Test = load_images("../dataset/archive", df_Test["Path"])

    # GET THE TOTAL UNIQUE TRANSIT SIGN
    num_labels = df_Train["ClassId"].unique().size
    
    # GET THE Y CLASS
    Y_Train = np.array(df_Train["ClassId"])
    Y_Test = np.array(df_Test["ClassId"])
    
    # NORMALIZE THE RGB PIXELS VALUES
    X_Train = X_Train.astype("float32") / 255.0
    X_Test  = X_Test.astype("float32") / 255.0
    
    # ONE HOT ENCODING
    Y_Train = to_categorical(Y_Train, num_labels)
    Y_Test = to_categorical(Y_Test, num_labels)  
    
    # CREATE A DICTIONARY BY CLASS AND CALC THE WEIGHT, THIS BECAUSE THE DATASET IS NOT HOMOGENEOUS
    # GET THE TOTAL TRANSIT SIGN IMAGES
    total_classes = Y_Train.sum(axis=0)
    class_weight = dict()
    
    # CALCULATE THE CLASS WEIGHT
    for i in range(0, len(total_classes)):
        class_weight[i] = total_classes.max() / total_classes[i]
        
    # IMAGE GENERATOR
    img_gen = ImageDataGenerator(rotation_range = 10,
                                 zoom_range = 0.15,
                                 width_shift_range = 0.1,
                                 height_shift_range = 0.1,
                                 shear_range = 0.15,
                                 horizontal_flip = False,
                                 vertical_flip = False,
                                 fill_mode = "nearest")
    
    print(f"[DEBUG] Config Model ...\n")
    # ADAM OPTIMIZER
    opt = Adam(learning_rate=INIT_LR, decay = INIT_LR / (NUM_EPOCHS * 0.5))
    
    # CNN
    model = build(width = 32, height = 32, depth = 3, classes = num_labels)
    model.compile(loss = "categorical_crossentropy", optimizer = opt, metrics = ["accuracy"])
    
    print(f"[DEBUG] Training ...\n")
    
    hist = model.fit(img_gen.flow(X_Train, Y_Train, batch_size = BS),
                    validation_data = (X_Test, Y_Test), 
                    steps_per_epoch = X_Train.shape[0] // BS,
                    epochs = NUM_EPOCHS,
                    class_weight = class_weight,
                    verbose = 1)
    
    print(f"[DEBUG] Evaluating ...\n")
    prediction = model.predict(X_Test, batch_size = BS)
    print(classification_report(Y_Test.argmax(axis=1), prediction.argmax(axis=1), target_names = label_names))
    
    # GETTING THE CURRENT TIME
    now = datetime.now()
    date = now.strftime("%d_%m_%Y_%H_%M_%S")
    
    # SAVING MODEL
    model_path = "../models/sign_model_" + date
    print(f"[INFO] serialiazing network to {model_path}")
    model.save(model_path)
    
    
    

In [117]:
run()

[DEBUG] Loading dataset ...

[DEBUG] Load images from file and prepare data ...



/Users/ivastival/opt/anaconda3/lib/python3.7/site-packages/skimage/util/dtype.py:135: UserWarning: Possible precision loss when converting from float64 to uint16
  .format(dtypeobj_in, dtypeobj_out))


[DEBUG] Total processed images: 1000
[DEBUG] Total processed images: 2000
[DEBUG] Total processed images: 3000
[DEBUG] Total processed images: 4000
[DEBUG] Total processed images: 5000
[DEBUG] Total processed images: 6000
[DEBUG] Total processed images: 7000
[DEBUG] Total processed images: 8000
[DEBUG] Total processed images: 9000
[DEBUG] Total processed images: 10000
[DEBUG] Total processed images: 11000
[DEBUG] Total processed images: 12000
[DEBUG] Total processed images: 13000
[DEBUG] Total processed images: 14000
[DEBUG] Total processed images: 15000
[DEBUG] Total processed images: 16000
[DEBUG] Total processed images: 17000
[DEBUG] Total processed images: 18000
[DEBUG] Total processed images: 19000
[DEBUG] Total processed images: 20000
[DEBUG] Total processed images: 21000
[DEBUG] Total processed images: 22000
[DEBUG] Total processed images: 23000
[DEBUG] Total processed images: 24000
[DEBUG] Total processed images: 25000
[DEBUG] Total processed images: 26000
[DEBUG] Total process

NameError: name 'label_names' is not defined

0                                  Speed limit (20km/h)
1                                  Speed limit (30km/h)
2                                  Speed limit (50km/h)
3                                  Speed limit (60km/h)
4                                  Speed limit (70km/h)
5                                  Speed limit (80km/h)
6                           End of speed limit (80km/h)
7                                 Speed limit (100km/h)
8                                 Speed limit (120km/h)
9                                            No passing
10         No passing for vechiles over 3.5 metric tons
11                Right-of-way at the next intersection
12                                        Priority road
13                                                Yield
14                                                 Stop
15                                          No vechiles
16             Vechiles over 3.5 metric tons prohibited
17                                             N